In [1]:
import Pkg; Pkg.activate("..")

  Activating project at `~/com.github/lucifer1004/AdventOfCode.jl`


In [2]:
using AdventOfCode

┌ Info: Precompiling AdventOfCode [ec480b64-8d68-453c-9e84-9dbf6ce847d6]
└ @ Base loading.jl:1664


In [3]:
year = 2019
day = 18

18

In [4]:
input = get_input(year, day)

"#################################################################################\n#.........#.......#...#...#..v..#.....#.#.........#...#....g#...L...#......z#...#\n#.#.#####.#.#####.###.#.#.#.#.###I#.#.#.#.#.#######E#.#.###.#.#####.#.#####.#.###\n#.#.....#...#.........#." ⋯ 6101 bytes ⋯ ".#.....#.#.Z.#...#...#.#\n#.#.###.###########.#.#######.#.#####.#.###.#.#######.#.###.###.#.#.#####.#.#.#.#\n#h....#..........e..#.........#.......#.#.............#.....#.....#.........#...#\n#################################################################################"

In [5]:
parsed_input = parse_input(input)

81-element Vector{SubString{String}}:
 "###############################" ⋯ 19 bytes ⋯ "###############################"
 "#.........#.......#...#...#..v." ⋯ 19 bytes ⋯ "#...#....g#...L...#......z#...#"
 "#.#.#####.#.#####.###.#.#.#.#.#" ⋯ 19 bytes ⋯ "#E#.#.###.#.#####.#.#####.#.###"
 "#.#.....#...#.........#.#...#.." ⋯ 19 bytes ⋯ "..#.#.#.#...#...#.#...#...#..p#"
 "#.#####.#####.#######.#.#######" ⋯ 19 bytes ⋯ "###.#.#.#####.#.#.###.#O###.#.#"
 "#.#...#.#.....#...#...#........" ⋯ 19 bytes ⋯ "..#.#.#.......#.#.....#.#...#.#"
 "#.###.#.#######.#.#.###########" ⋯ 19 bytes ⋯ "#.#.#.#.#####.#########.#####.#"
 "#...#.#..y..#...#.#...#........" ⋯ 19 bytes ⋯ "#.#...#.#...#.........#.#.....#"
 "###.#.#####.#.###.#####.#######" ⋯ 19 bytes ⋯ "#.#####.#.#.###.#.#####.#.#####"
 "#...#....c#...#.#.......#......" ⋯ 19 bytes ⋯ "#....x#.#.#.#...#.#.....#.....#"
 "#.###.#.#######.#########.#####" ⋯ 19 bytes ⋯ "#####.###.#.#.#####.#####.###.#"
 "#.#...#.................#.....#" ⋯ 19 bytes ⋯ "..#.#...#.#

In [6]:
sample = """#########
#b.A.@.a#
#########"""

"#########\n#b.A.@.a#\n#########"

In [7]:
parsed_sample = parse_input(sample)

3-element Vector{SubString{String}}:
 "#########"
 "#b.A.@.a#"
 "#########"

In [8]:
DIRS = [(0, 1), (0, -1), (1, 0), (-1, 0)]

4-element Vector{Tuple{Int64, Int64}}:
 (0, 1)
 (0, -1)
 (1, 0)
 (-1, 0)

In [9]:
function part_one(input)
    n = length(input)
    m = length(input[1])

    si, sj = 0, 0
    for i in 1:n
        for j in 1:m
            if input[i][j] == '@'
                si, sj = i, j
            end
        end
    end

    pre = zeros(Int, 27)
    dch = zeros(Int, 27)
    dist = fill(-1, n, m)
    dist[si, sj] = 0
    pos = fill((0, 0), 27)
    function dfs(i, j, doors)
        for (di, dj) in DIRS
            ni, nj = i + di, j + dj
            if ni < 1 || ni > n || nj < 1 || nj > m
                continue
            end
            if dist[ni, nj] != -1
                continue
            end
            ch = input[ni][nj]
            if ch == '#'
                continue
            end
            dist[ni, nj] = dist[i, j] + 1
            doors′ = doors
            if 'A' <= ch <= 'Z'
                doors′ |= 1 << (ch - 'A')
            end
            if 'a' <= ch <= 'z'
                pre[ch - 'a' + 1] = doors
                dch[ch - 'a' + 1] = dist[ni, nj]
                pos[ch - 'a' + 1] = (ni, nj)
            end
            dfs(ni, nj, doors′)
        end
    end

    dfs(si, sj, 0)
    k = findfirst((==)(0), dch) - 1
    adj = fill(-1, k, k)
    for i in 1:k
        adj[i, i] = 0
        fill!(dist, -1)
        dist[pos[i]...] = 0
        q = Queue{Tuple{Int, Int}}()
        enqueue!(q, pos[i])
        while !isempty(q)
            ci, cj = dequeue!(q)
            for (di, dj) in DIRS
                ni, nj = ci + di, cj + dj
                if ni < 1 || ni > n || nj < 1 || nj > m
                    continue
                end
                if dist[ni, nj] != -1
                    continue
                end
                ch = input[ni][nj]
                if ch == '#'
                    continue
                end

                dist[ni, nj] = dist[ci, cj] + 1
                if 'a' <= ch <= 'z'
                    adj[i, ch - 'a' + 1] = dist[ni, nj]
                end
                enqueue!(q, (ni, nj))
            end
        end
    end

    msk = (1 << k) - 1
    dp = fill(Int(1e9), k, msk)
    for i in 1:k
        if pre[i] == 0
            dp[i, 1 << (i - 1)] = dch[i]
        end
    end

    for state in 1:msk - 1
        for i in 1:k
            if dp[i, state] == Int(1e9)
                continue
            end
            for j in 1:k
                if (state >> (j - 1)) & 1 == 1
                    continue
                end
                if (pre[j] & state) != pre[j]
                    continue
                end
                dp[j, state | (1 << (j - 1))] = min(dp[j, state | (1 << (j - 1))], dp[i, state] + adj[i, j])
            end
        end
    end

    return minimum(dp[:, msk])
end

part_one (generic function with 1 method)

In [10]:
part_one(parsed_sample)

8

In [11]:
part_one(parse_input("""########################
#f.D.E.e.C.b.A.@.a.B.c.#
######################.#
#d.....................#
########################"""))

86

In [12]:
part_one(parse_input("""########################
#...............b.C.D.f#
#.######################
#.....@.a.B.c.d.A.e.F.g#
########################"""))

132

In [13]:
part_one_ans = part_one(parsed_input)

4668

In [14]:
submit_answer(year, day, part_one_ans)

"<!DOCTYPE html>\n<html lang=\"en-us\">\n<head>\n<meta charset=\"utf-8\"/>\n<title>Day 18 - Advent of Code 2019</title>\n<!--[if lt IE 9]><script src=\"/static/html5.js\"></script><![endif]-->\n<link href='//fonts.googleapis.com/css?family=Source+Code+Pro:300&subset=latin,latin-ext'" ⋯ 3177 bytes ⋯ "Name(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)\n})(window,document,'script','//www.google-analytics.com/analytics.js','ga');\nga('create', 'UA-69522494-1', 'auto');\nga('set', 'anonymizeIp', true);\nga('send', 'pageview');\n</script>\n<!-- /ga -->\n</body>\n</html>"

In [15]:
function part_two(input)
    input = collect.(input)
    n = length(input)
    m = length(input[1])

    si, sj = 0, 0
    for i in 1:n
        for j in 1:m
            if input[i][j] == '@'
                si, sj = i, j
            end
        end
    end

    input[si-1][sj-1:sj+1] .= collect("@#@")
    input[si][sj-1:sj+1] .= collect("###")
    input[si+1][sj-1:sj+1] .= collect("@#@")

    pre = zeros(Int, 27)
    dch = zeros(Int, 27)
    dist = fill(-1, n, m)
    dist[si-1, sj-1] = dist[si-1, sj+1] = dist[si+1, sj-1] = dist[si+1, sj+1] = 0
    pos = fill((0, 0), 27)
    vaults = [Int[] for _ in 1:4]
    function dfs(i, j, doors, vault)
        for (di, dj) in DIRS
            ni, nj = i + di, j + dj
            if ni < 1 || ni > n || nj < 1 || nj > m
                continue
            end
            if dist[ni, nj] != -1
                continue
            end
            ch = input[ni][nj]
            if ch == '#'
                continue
            end
            dist[ni, nj] = dist[i, j] + 1
            doors′ = doors
            if 'A' <= ch <= 'Z'
                doors′ |= 1 << (ch - 'A')
            end
            if 'a' <= ch <= 'z'
                pre[ch-'a'+1] = doors
                dch[ch-'a'+1] = dist[ni, nj]
                pos[ch-'a'+1] = (ni, nj)
                push!(vaults[vault], ch - 'a' + 1)
            end
            dfs(ni, nj, doors′, vault)
        end
    end

    dfs(si - 1, sj - 1, 0, 1)
    dfs(si - 1, sj + 1, 0, 2)
    dfs(si + 1, sj - 1, 0, 3)
    dfs(si + 1, sj + 1, 0, 4)
    k = findfirst((==)(0), dch) - 1

    adj = fill(Int(1e9), k, k)
    for i in 1:k
        adj[i, i] = 0
        fill!(dist, -1)
        dist[pos[i]...] = 0
        q = Queue{Tuple{Int,Int}}()
        enqueue!(q, pos[i])
        while !isempty(q)
            ci, cj = dequeue!(q)
            for (di, dj) in DIRS
                ni, nj = ci + di, cj + dj
                if ni < 1 || ni > n || nj < 1 || nj > m
                    continue
                end
                if dist[ni, nj] != -1
                    continue
                end
                ch = input[ni][nj]
                if ch == '#'
                    continue
                end

                dist[ni, nj] = dist[ci, cj] + 1
                if 'a' <= ch <= 'z'
                    adj[i, ch-'a'+1] = dist[ni, nj]
                end
                enqueue!(q, (ni, nj))
            end
        end
    end

    msk = (1 << k) - 1
    best = Int(1e9)
    history = Dict{Tuple{Int,Int,Int,Int,Int}, Int}()
    function dfs2(state, last, steps)
        curr = (state, last[1], last[2], last[3], last[4])
        if get(history, curr, Int(1e9)) <= steps || steps >= best
            return
        end
        history[curr] = steps

        if state == msk
            best = steps
            return
        end

        for i in 1:4
            for v in vaults[i]
                if (state >> (v - 1)) & 1 == 1
                    continue
                end
                if (pre[v] & state) != pre[v]
                    continue
                end

                dis = last[i] == -1 ? dch[v] : adj[last[i], v]
                tmp = last[i]
                last[i] = v
                dfs2(state | (1 << (v - 1)), last, steps + dis)
                last[i] = tmp
            end
        end
    end

    dfs2(0, [-1, -1, -1, -1], 0)

    return best
end

part_two (generic function with 1 method)

In [16]:
part_two(parse_input("""#######
#a.#Cd#
##...##
##.@.##
##...##
#cB#Ab#
#######"""))

8

In [17]:
part_two(parse_input("""###############
#d.ABC.#.....a#
######...######
######.@.######
######...######
#b.....#.....c#
###############"""))

24

In [18]:
part_two_ans = part_two(parsed_input)

1910

In [19]:
submit_answer(year, day, part_two_ans, 2)

"<!DOCTYPE html>\n<html lang=\"en-us\">\n<head>\n<meta charset=\"utf-8\"/>\n<title>Day 18 - Advent of Code 2019</title>\n<!--[if lt IE 9]><script src=\"/static/html5.js\"></script><![endif]-->\n<link href='//fonts.googleapis.com/css?family=Source+Code+Pro:300&subset=latin,latin-ext'" ⋯ 3332 bytes ⋯ "Name(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)\n})(window,document,'script','//www.google-analytics.com/analytics.js','ga');\nga('create', 'UA-69522494-1', 'auto');\nga('set', 'anonymizeIp', true);\nga('send', 'pageview');\n</script>\n<!-- /ga -->\n</body>\n</html>"